In [1]:
import syft as sy
from syft.spdz import spdz
from syft.mpc.securenn import decompose, private_compare, msb, relu_deriv
from syft.core.frameworks.torch.tensor import _GeneralizedPointerTensor, _SPDZTensor, _SNNTensor
from syft.core.frameworks.torch import utils as torch_utils

import unittest
import numpy as np
import torch
import importlib

hook = sy.TorchHook(verbose=True)

me = hook.local_worker
me.is_client_worker = False

bob = sy.VirtualWorker(id="bob", hook=hook, is_client_worker=False)
alice = sy.VirtualWorker(id="alice", hook=hook, is_client_worker=False)

me.add_workers([bob, alice])
bob.add_workers([me, alice])
alice.add_workers([me, bob])

In [4]:
def egcd(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = egcd(b % a, a)
        return (g, x - (b // a) * y, y)

def modinv(a, m):
    g, x, y = egcd(a, m)
    if g != 1:
        raise Exception('modular inverse does not exist')
    else:
        return x % m

BASE = 10
PRECISION_INTEGRAL = 1
PRECISION_FRACTIONAL = 3
Q = spdz.field

PRECISION = PRECISION_INTEGRAL + PRECISION_FRACTIONAL
INVERSE = modinv(BASE**PRECISION_FRACTIONAL, Q) # inverse of BASE**FRACTIONAL_PRECISION
# there’s 1/2**kappa chance that in one of the steps in the
# protocol you’re revealing something about the private values
KAPPA = 1 # leave room for five digits overflow before leakage

assert((INVERSE * BASE**PRECISION_FRACTIONAL) % Q == 1)
assert(Q > BASE**(2*PRECISION + KAPPA))
assert(Q > BASE**PRECISION)

In [18]:
x = torch.FloatTensor([1,-1,-2,3, 4, 0.5, 0.6, 0.7, 0.8, 0.9]).fix_precision()

In [19]:
y = x * x

In [20]:
# x = torch.FloatTensor([0.1,-0.1,-0.2,0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]).fix_precision()
x = torch.FloatTensor([1,-1,-2,3, 4, 0.5, 0.6, 0.7, 0.8, 0.9]).fix_precision()

a = x.share(alice,bob)
a = ((a.child.child * a.child.child))

In [21]:
def truncate(a):

    global BASE
    
    b = a + (BASE**(2*PRECISION+1))

    rand_shape = torch.IntTensor(list(b.get_shape())).prod()
    mask = torch.LongTensor(1).send(bob).expand(rand_shape).contiguous().random_(BASE**PRECISION)
    mask_low = torch.fmod(mask,BASE**PRECISION_FRACTIONAL)
    mpc_mask = mask.share(bob, alice).get()
    b_masked = (b + mpc_mask).get()
    b_masked_low = torch.fmod(b_masked,BASE**PRECISION_FRACTIONAL)
    b_low = b_masked_low.share(bob, alice) - mask_low.share(bob, alice).get()
    c = (a - b_low) * INVERSE
    return c

In [22]:
sy._FixedPrecisionTensor(child=truncate(a).get(), already_encoded=True).decode()


  1.0000
  1.0000
  4.0000
  9.0000
 16.0000
  0.2500
  0.3600
  0.4900
  0.6400
  0.8100
[syft.core.frameworks.torch.tensor.FloatTensor of size 10]

In [6]:
a = (x.child.child * x.child.child)

In [5]:
a % (10**3)

[Head of chain]
[syft.core.frameworks.torch.tensor.LongTensor with no dimension]

In [6]:
spdz.field

2147483648

In [7]:
spdz.field

2147483648

In [14]:
(2 ** 31) - 1

2147483647

In [12]:
spdz.field

2147483648

In [11]:
((b * modinv(1000, spdz.field)) % (spdz.field)).get()

Exception: modular inverse does not exist

In [130]:

# # TODO: generate this mask on P0
# mask = (torch.rand(list(a.get_shape())) * spdz.field).long()
# mask_low = mask % (BASE**PRECISION + KAPPA)
# b_masked = (b + mask.share(alice,bob)).get()
# b_masked_low = b_masked % (BASE**PRECISION)
# b_low = b_masked_low.share(bob, alice) - mask_low.share(alice, bob)
# c = a - b_low
# c.get()

In [2]:
x = (torch.LongTensor([[-1,3,-5,7],[-1,0,1,2]])).share(alice,bob)
y = (torch.LongTensor([[-2,4,-6,8],[-2,0,1,2]])).share(alice,bob)

In [3]:
(x > y).get()


 1  0  1  0
 1  0  0  0
[syft.core.frameworks.torch.tensor.LongTensor of size 2x4]

In [4]:
(x < y).get()


 0  1  0  1
 0  0  0  0
[syft.core.frameworks.torch.tensor.LongTensor of size 2x4]

In [5]:
(x >= y).get()


 1  0  1  0
 1  1  1  1
[syft.core.frameworks.torch.tensor.LongTensor of size 2x4]

In [6]:
(x <= y).get()


 0  1  0  1
 0  1  1  1
[syft.core.frameworks.torch.tensor.LongTensor of size 2x4]

In [7]:
(x == y).get()


 0  0  0  0
 0  1  1  1
[syft.core.frameworks.torch.tensor.LongTensor of size 2x4]

In [5]:
v = sy.FloatTensor([1.1])
v2 = sy.FloatTensor([1.2])
v.__gt__(v2)


 0
[syft.core.frameworks.torch.tensor.ByteTensor of size 1]

In [3]:
# def newgt(self, *args, **kwargs):
#     try:
#         out = self.child > args[0].child
#         return out
#     except:
#         return self.native___gt__(*args, **kwargs)

In [4]:
# # torch.LongTensor.native___gt__ = torch.LongTensor.__gt__
# torch.LongTensor.__gt__ = newgt

In [5]:
x > y

RuntimeError: Command "gt" is not a supported Torch operation.

In [5]:
x.child > y.child

executing
running relu
running relu


[Head of chain]
[syft.core.frameworks.torch.tensor.LongTensor with no dimension]

In [6]:
(x.child > y.child).get()

executing
running relu
running relu



 1  0  1  0
 1  0  0  0
[syft.core.frameworks.torch.tensor.LongTensor of size 2x4]

In [8]:
out = x.relu()

running relu
running relu


In [6]:
out.get()


 0  3  0  7
 0  0  1  2
[syft.core.frameworks.torch.tensor.LongTensor of size 2x4]